# Build jsons for timed experiment

## Json format:

```
{

"level1":[
            {url:"url1", time: 100},  # time in msec
            {url:"url2", time: 3000},
            
            ...
         ],
"level2":[
            {url:"url1", time: 100},
            {url:"url2", time: 3000},
            ...
            
         ],
"level3":[
            {url:"url1", time: 100},
            {url:"url2", time: 3000},
            ...
            
         ]

.
.
.

}
```




## Structure of the jsons

- 50% real, 50% fake (balanced prevalence), but also 15% fake (low prevalence)
- Control the fakeness of deepfakes (work with elements that don't have obvious artifacts)
- Create versions with and without caricatures
- Same participant across different times
- Practice trials
- In each level, different times, but randomize order
- Balance the number of videos shown, number of times within a level, times shown per video
- Make sure we show caricatures and fakes an equal amount of times
- Mix both caricatures and deepfakes in one level

In [1]:
# Imports
import numpy as np
import matplotlib.pyplot as plt
import json
import os

In [2]:
# Load list of real deepfake URLs
def load_urls_from_jsons(json_folder):
    df_urls = set()
    json_folder = './exp2_jsons'
    for f in os.listdir(json_folder):
        if not f.endswith('.json'): continue
        j = json.load(open(os.path.join(json_folder,f)))
        for k in j.keys():
            for u in j[k]:
                df_urls.add(u)

    print("\nLoaded %d deepfake urls" % len(df_urls))

    return df_urls


vids_for_exp2_5levels_id8.json
vids_for_exp2_5levels_id6.json
vids_for_exp2_5levels_id5.json
vids_for_exp2_5levels_id2.json
vids_for_exp2_5levels_id4.json
new_exp2_5levels_id9.json
new_exp2_5levels_id3.json
vids_for_exp2_5levels_id0.json
new_exp2_5levels_id2.json
new_exp2_5levels_id7.json
new_exp2_5levels_id5.json
new_exp2_5levels_id8.json
vids_for_exp2_5levels_id9.json
new_exp2_5levels_id1.json
vids_for_exp2_5levels_id7.json
vids_for_exp2_5levels_id1.json
new_exp2_5levels_id0.json
vids_for_exp2_5levels_id10.json
new_exp2_5levels_id4.json
new_exp2_5levels_id6.json
vids_for_exp2_5levels_id3.json

Loaded 1015 deepfake urls


In [3]:
show_urls()

NameError: name 'show_urls' is not defined

In [ ]:
# Build Jsons

def build_json(urls, times):
    